In [1]:
from mmengine import __version__
from mmcv import __version__ as mmcv_version
from mmdet import __version__ as mmdet_version
from torch import __version__ as torch_version
print(torch_version)
print(f"mmengine version: {__version__}")
print(f"mmcv version: {mmcv_version}")
print(f"mmdet version: {mmdet_version}")

2.1.0+cu118
mmengine version: 0.10.3
mmcv version: 2.1.0
mmdet version: 3.3.0


In [2]:
from mmdet.utils import collect_env
collect_env()

OrderedDict([('sys.platform', 'linux'),
             ('Python', '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'),
             ('CUDA available', True),
             ('MUSA available', False),
             ('numpy_random_seed', 2147483648),
             ('GPU 0', 'NVIDIA GeForce RTX 3060 Laptop GPU'),
             ('CUDA_HOME', '/usr/local/cuda'),
             ('NVCC', 'Cuda compilation tools, release 11.8, V11.8.89'),
             ('GCC',
              'x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0'),
             ('PyTorch', '2.1.0+cu118'),
             ('PyTorch compiling details',
              'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201703\n  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  -

In [1]:
from mmengine.config import Config

cfg = Config.fromfile('mmdetection/configs/retinanet/retinanet_x101-64x4d_fpn_1x_coco.py')
cfg.data_root = "mmdetection/data/boxes/"
print(cfg.data_root)

mmdetection/data/boxes/


In [2]:
cfg.model.bbox_head.num_classes = 1
cfg.data_root = "mmdetection/data/boxes/"

metainfo = {
    'classes': ('Box', ),
    'palette': [
        (220, 20, 60),
    ]
}

cfg.train_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='train.json',
        data_prefix=dict(img='train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=cfg.train_pipeline,
        backend_args=cfg.backend_args))
cfg.val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='val.json',
        data_prefix=dict(img='valid/'),
        test_mode=True,
        pipeline=cfg.test_pipeline,
        backend_args=cfg.backend_args))

cfg.val_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'val.json',
    metric='bbox',
    format_only=False,
    backend_args=cfg.backend_args)


# inference on test dataset and
# format the output results for submission.
cfg.test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file=cfg.data_root + 'test.json',
        data_prefix=dict(img='test/'),
        test_mode=True,
        pipeline=cfg.test_pipeline))
cfg.test_evaluator = dict(
    type='CocoMetric',
    metric='bbox',
    format_only=True,
    ann_file=cfg.data_root + 'test.json',
    outfile_prefix='./work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/test')



# cfg.param_scheduler = list(
# cfg.param_scheduler =  dict(type='LinearLR', start_factor=0.001, by_epoch=False, begin=0, end=1)
#     dict(
#         type='MultiStepLR',
#         begin=0,
#         end=1,
#         by_epoch=True,
#         milestones=[8, 11],
#         gamma=0.1)
# )

cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=12, val_interval=1)
# cfg.param_scheduler[1].end = 2
# cfg.param_scheduler[0].end = 2
# cfg.param_scheduler[0].by_epoch = True
cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='SGD', lr=0.0002, momentum=0.9, weight_decay=0)
    # optimizer=dict(type='Adam', lr=0.0002)
    ,)
cfg.auto_scale_lr = dict(enable=False, base_batch_size=1)
cfg.work_dir = './work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/1'

cfg.default_hooks = dict(
                checkpoint=dict(
                    type='CheckpointHook', 
                    interval=3,
                    max_keep_ckpts=2,  # only keep latest 2 checkpoints
                    save_best='auto'
                    ),
                logger=dict(type='LoggerHook', interval=10),
                
                visualization=dict(type='DetVisualizationHook'),
                )
# cfg.vis_backends = dict(type='LocalVisBackend')


In [3]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

runner.train()

03/08 17:14:39 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1359811170
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencod

RetinaNet(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): ResNeXt(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu):

In [ ]:
cfg.dump('/home/jayant/Projects/mmdetection/latest/retinanet_x101-64x4d_fpn_1x_boxes.py')

In [3]:
from mmdet.apis import DetInferencer

model = DetInferencer('/home/jayant/Projects/mmdetection/latest/retinanet_x101-64x4d_fpn_1x_boxes.py',"work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/1/best_coco_bbox_mAP_epoch_11.pth")

Loads checkpoint by local backend from path: work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/1/best_coco_bbox_mAP_epoch_11.pth
03/09 13:30:24 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/home/jayant/Projects/mmdetection/latest/.venv/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [5]:
pred = model("/home/jayant/Projects/mmdetection/latest/images/44.jpg",out_dir="/home/jayant/Projects/mmdetection/latest/outputs/2",  )

Output()

In [16]:
!python mmdetection/tools/analysis_tools/analyze_logs.py plot_curve /home/jayant/Projects/mmdetection/latest/work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/1/20240308_171438/vis_data/20240308_171438.json --keys loss --legend loss   --out /home/jayant/Projects/mmdetection/latest/records/retinanet_x101-64x4d_fpn_1x_boxes/1/loss.jpg --title "retinanet/total_loss"

plot curve of /home/jayant/Projects/mmdetection/latest/work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/1/20240308_171438/vis_data/20240308_171438.json, metric is loss
save curve to: /home/jayant/Projects/mmdetection/latest/records/retinanet_x101-64x4d_fpn_1x_boxes/1/loss.jpg


In [6]:
!python mmdetection/tools/test.py /home/jayant/Projects/mmdetection/latest/retinanet_x101-64x4d_fpn_1x_boxes.py /home/jayant/Projects/mmdetection/latest/work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/1/best_coco_bbox_mAP_epoch_11.pth --out result_boxes_retinanet_pkl/retinanet/result_1.pkl --cfg-options "jsonfile_prefix=./result_box_retinanet" 

03/09 13:33:35 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 556540526
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode

In [7]:
!python mmdetection/tools/analysis_tools/analyze_results.py /home/jayant/Projects/mmdetection/latest/retinanet_x101-64x4d_fpn_1x_boxes.py /home/jayant/Projects/mmdetection/latest/result_boxes_retinanet_pkl/retinanet/result_1.pkl result_boxes_retinanet --show 

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
03/09 13:34:38 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1450423038
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CP

In [8]:
!python mmdetection/tools/analysis_tools/confusion_matrix.py /home/jayant/Projects/mmdetection/latest/retinanet_x101-64x4d_fpn_1x_boxes.py /home/jayant/Projects/mmdetection/latest/result_boxes_retinanet_pkl/retinanet/result_1.pkl  result_boxes_retinanet/ 

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 767.3 task/s, elapsed: 0s, ETA:     0s/home/jayant/Projects/mmdetection/latest/mmdetection/tools/analysis_tools/confusion_matrix.py:229: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()


In [9]:
!python mmdetection/tools/analysis_tools/eval_metric.py /home/jayant/Projects/mmdetection/latest/retinanet_x101-64x4d_fpn_1x_boxes.py /home/jayant/Projects/mmdetection/latest/result_boxes_retinanet_pkl/retinanet/result_1.pkl

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
03/09 13:37:47 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.60s).
Accumulating evaluation results...
DONE (t=0.11s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.006
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.032
 A

In [10]:
!python mmdetection/tools/analysis_tools/coco_error_analysis.py work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/test.bbox.json results --ann /home/jayant/Projects/mmdetection/latest/mmdetection/data/boxes/test.json 

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.02s).
--------------analyzing 1-Box---------------
creating index...
index created!
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/home/jayant/Projects/mmdetection/latest/mmdetection/tools/analysis_tools/coco_error_analysis.py", line 195, in analyze_individual_category
    child_catIds = gt.getCatIds(supNms=[nm['supercategory']])
KeyError: 'supercategory'
"""

The above exception was the direct cause of the following exce

In [11]:
!python /home/jayant/Projects/mmdetection/latest/mmdetection/tools/analysis_tools/robustness_eval.py /home/jayant/Projects/mmdetection/latest/work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/test.bbox.json

Traceback (most recent call last):
  File "/home/jayant/Projects/mmdetection/latest/mmdetection/tools/analysis_tools/robustness_eval.py", line 263, in <module>
    main()
  File "/home/jayant/Projects/mmdetection/latest/mmdetection/tools/analysis_tools/robustness_eval.py", line 253, in main
    get_results(
  File "/home/jayant/Projects/mmdetection/latest/mmdetection/tools/analysis_tools/robustness_eval.py", line 177, in get_results
    results = get_coco_style_results(
  File "/home/jayant/Projects/mmdetection/latest/mmdetection/tools/analysis_tools/robustness_eval.py", line 74, in get_coco_style_results
    num_distortions = len(list(eval_output.keys()))
AttributeError: 'list' object has no attribute 'keys'


In [18]:
!python mmdetection/tools/analysis_tools/benchmark.py /home/jayant/Projects/mmdetection/latest/retinanet_x101-64x4d_fpn_1x_boxes.py --checkpoint /home/jayant/Projects/mmdetection/latest/work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/1/best_coco_bbox_mAP_epoch_11.pth --repeat-num 5 --task inference --work-dir /home/jayant/Projects/mmdetection/latest/records/retinanet_x101-64x4d_fpn_1x_boxes/1/ --dataset-type train

03/09 14:00:10 - mmengine - INFO - before build: 
03/09 14:00:10 - mmengine - INFO - (GB) mem_used: 5.87 | uss: 0.31 | pss: 0.40 | total_proc: 1
Loads checkpoint by local backend from path: /home/jayant/Projects/mmdetection/latest/work_dirs/retinanet_x101-64x4d_fpn_1x_boxes/1/best_coco_bbox_mAP_epoch_11.pth
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
03/09 14:00:12 - mmengine - INFO - after build: 
03/09 14:00:13 - mmengine - INFO - (GB) mem_used: 6.29 | uss: 0.72 | pss: 0.82 | total_proc: 1
03/09 14:00:22 - mmengine - INFO - ==================================
03/09 14:00:22 - mmengine - INFO - Done image [50 /2000], fps: 5.3 img/s, times per image: 187.7 ms/img, cuda memory: 586 MB
03/09 14:00:22 - mmengine - INFO - (GB) mem_used: 6.35 | uss: 0.79 | pss: 0.95 | total_proc: 1
03/09 14:00:35 - mmengine - INFO - ==================================
03/09 14:00:35 - mmengine - INFO - Done image [50 /2000], fps: 5.3 img/s, times per image: 189.6 ms/img,